# <font color ='blue'>Zillow Clustering Project - Working Copy
    
<img src="zillow_logo.png" alt="Zillow Logo" title="Zillow Logo" width="300" height="100" align="right"/>
    
    
# <p style="font-size:50px" align="center">Zestimate Error</p> <sub>By Kaniela Denis - 13 Dec 2021</sub>

In [1]:
# import pandas and numpy to start coding
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats

# prep methods
from sklearn.model_selection import train_test_split

# Scaler
from sklearn.preprocessing import MinMaxScaler

# cluster method
from sklearn.cluster import KMeans

# Feature Engineering methods
import sklearn.linear_model
import sklearn.feature_selection
import sklearn.preprocessing

# Modeling methods
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures


import warnings
warnings.filterwarnings("ignore")

<hr style="border:2px solid blue"> </hr>

# <font color = 'red'>Planning

### Project Goals

- Predict logerror of Zestimate
- Discover the drivers of the error in the Zcestimate
- Compare four different clustering models
- Conclude if clustering helps with discovering drivers
- 


- A model that predicts property tax assessed values of single family properties based on 2017 transactions.
- Identify ways to make a better model for Zillow.
- Create new features out of existing features.
- Try non-linear regression model or try a model for each county.


### Project Description

Deliver:
1. Github Repo
    - README.md
    - Final Report
    - Working Notebook
    - Modules (Acquire, Prepare, Model)
    - Misc Artifacts
2. Further Requirements:
    - Aquistion:
        - SQL Query
    - Prep:
        - Columns have appropriate Data Types
        - Missing values and Outliers are investigated/handled
    - Explore:
        - Use vizulization and statistical testing for target/independ variable interations
        - Clustering is used to explore the data. Minumum of 3 combinations of features tried. A conclusion is made on whether clustering helped, supported by visualizations and statistical testing.
    - Modeling: 
        - Minimum 4 different models created and performance compared
3. Final Notebook with Recommendation
    - 4 Visuals, 2 with stats tests
        - Data Context of Target Variable (Distribution of values)
    - 3 Best Models (show steps/code to fit/eval/select)
    - Visual of how best Model performed
4. Working Notebook:
    - Reproducible comments
    - Exploration with at least two stats test during exploration
5. Presentation
    - Findings
    - Methodologies
    - Conclusions

### Initial Questions

- What are the transactions are in 2017?
- What states and counties are the properties located in?
- What is the tax rate per county or fips? (might have to combine columns to calcualate)
- What is the distribution of tax rates for each county?
- What is the distribution of taxes across fips?
- What are the drivers of single family property values?
- Why do some properties have a much higher value than others when they are located so close to each other?
- Why are some properties valued so differently from others when they have nearly the same physical attributes but only differ in location? 
- Is having 1 bathroom worse than having 2 bedrooms?

### Data Dictionary (Add to README)

In [2]:
# Read Data Dictionary from working directory
dd = pd.read_excel('zillow_data_dictionary.xlsx')

In [3]:
## References are available on my GitHub 

In [4]:
#[click me](https://github.com/kanieladenis/zillow_regression_project)

# <font color = 'red'> Acquisition (Get and Clean Data)

## <font color = 'red'>Get Data

- Added env.py to the repo .gitignore file so env.py won't be pushed to github
- Created an env file that contains my username, password, and host address fo the MySQL Server.

In [5]:
# import env file for hostname, username, password, and db_name
from env import host, user, password, db_name

In [6]:
# Pass env file authentication to container 'url'
url = f'mysql+pymysql://{user}:{password}@{host}/{db_name}'

### Pull records of houses with transactions from 2017

In [7]:
# define sql search for all records from all tables
sql = """
SELECT *
FROM properties_2017
LEFT JOIN predictions_2017 pred USING(parcelid)
LEFT JOIN airconditioningtype USING(airconditioningtypeid)
LEFT JOIN architecturalstyletype USING(architecturalstyletypeid)
LEFT JOIN buildingclasstype USING(buildingclasstypeid)
LEFT JOIN heatingorsystemtype USING(heatingorsystemtypeid)
LEFT JOIN propertylandusetype USING(propertylandusetypeid)
LEFT JOIN storytype USING(storytypeid)
LEFT JOIN typeconstructiontype USING(typeconstructiontypeid)
WHERE latitude IS NOT NULL 
AND longitude IS NOT NULL
AND propertylandusetypeid = 261
AND transactiondate LIKE "2017%%"
AND pred.id IN (SELECT MAX(id)
FROM predictions_2017
GROUP BY parcelid
HAVING MAX(transactiondate))
"""

### Note:
- ????

### Create Dataframe

In [8]:
# load zillow data from saved csv or pull from sql server and save to csv
import os
file = 'zillow_data.csv'
if os.path.isfile(file):
    df = pd.read_csv(file, index_col=0)
else:
    df = pd.read_sql(sql,url)
    df.to_csv(file)
df.head()

,typeconstructiontypeid,storytypeid,propertylandusetypeid,heatingorsystemtypeid,buildingclasstypeid,architecturalstyletypeid,airconditioningtypeid,parcelid,id,basementsqft,...,id.1,logerror,transactiondate,airconditioningdesc,architecturalstyledesc,buildingclassdesc,heatingorsystemdesc,propertylandusedesc,storydesc,typeconstructiondesc
0,NaN,NaN,261.0,NaN,NaN,NaN,NaN,14297519,1727539,NaN,...,0,0.025595,2017-01-01,NaN,NaN,NaN,NaN,Single Family Residential,NaN,NaN
1,NaN,NaN,261.0,NaN,NaN,NaN,NaN,17052889,1387261,NaN,...,1,0.055619,2017-01-01,NaN,NaN,NaN,NaN,Single Family Residential,NaN,NaN
2,NaN,NaN,261.0,NaN,NaN,NaN,NaN,14186244,11677,NaN,...,2,0.005383,2017-01-01,NaN,NaN,NaN,NaN,Single Family Residential,NaN,NaN
3,NaN,NaN,261.0,2.0,NaN,NaN,NaN,12177905,2288172,NaN,...,3,-0.103410,2017-01-01,NaN,NaN,NaN,Central,Single Family Residential,NaN,NaN
4,NaN,NaN,261.0,2.0,NaN,NaN,1.0,12095076,781532,NaN,...,6,-0.001011,2017-01-01,Central,NaN,NaN,Central,Single Family Residential,NaN,NaN


In [9]:
def acquire_zillow():
    # import env file for hostname, username, password, and db_name
    from env import host, user, password, db_name

    # Pass env file authentication to container 'url'
    url = f'mysql+pymysql://{user}:{password}@{host}/{db_name}'

    # define sql search for all records from all tables
    sql = """
    SELECT *
    FROM properties_2017
    LEFT JOIN predictions_2017 pred USING(parcelid)
    LEFT JOIN airconditioningtype USING(airconditioningtypeid)
    LEFT JOIN architecturalstyletype USING(architecturalstyletypeid)
    LEFT JOIN buildingclasstype USING(buildingclasstypeid)
    LEFT JOIN heatingorsystemtype USING(heatingorsystemtypeid)
    LEFT JOIN propertylandusetype USING(propertylandusetypeid)
    LEFT JOIN storytype USING(storytypeid)
    LEFT JOIN typeconstructiontype USING(typeconstructiontypeid)
    WHERE latitude IS NOT NULL 
    AND longitude IS NOT NULL
    AND propertylandusetypeid = 261
    AND transactiondate LIKE "2017%%"
    AND pred.id IN (SELECT MAX(id)
    FROM predictions_2017
    GROUP BY parcelid
    HAVING MAX(transactiondate))
    """

    # load zillow data from saved csv or pull from sql server and save to csv
    import os
    file = 'zillow_data.csv'
    if os.path.isfile(file):
        df = pd.read_csv(file, index_col=0)
    else:
        df = pd.read_sql(sql,url)
        df.to_csv(file)
    return df

In [10]:
df = acquire_zillow()
df.head()

,typeconstructiontypeid,storytypeid,propertylandusetypeid,heatingorsystemtypeid,buildingclasstypeid,architecturalstyletypeid,airconditioningtypeid,parcelid,id,basementsqft,...,id.1,logerror,transactiondate,airconditioningdesc,architecturalstyledesc,buildingclassdesc,heatingorsystemdesc,propertylandusedesc,storydesc,typeconstructiondesc
0,NaN,NaN,261.0,NaN,NaN,NaN,NaN,14297519,1727539,NaN,...,0,0.025595,2017-01-01,NaN,NaN,NaN,NaN,Single Family Residential,NaN,NaN
1,NaN,NaN,261.0,NaN,NaN,NaN,NaN,17052889,1387261,NaN,...,1,0.055619,2017-01-01,NaN,NaN,NaN,NaN,Single Family Residential,NaN,NaN
2,NaN,NaN,261.0,NaN,NaN,NaN,NaN,14186244,11677,NaN,...,2,0.005383,2017-01-01,NaN,NaN,NaN,NaN,Single Family Residential,NaN,NaN
3,NaN,NaN,261.0,2.0,NaN,NaN,NaN,12177905,2288172,NaN,...,3,-0.103410,2017-01-01,NaN,NaN,NaN,Central,Single Family Residential,NaN,NaN
4,NaN,NaN,261.0,2.0,NaN,NaN,1.0,12095076,781532,NaN,...,6,-0.001011,2017-01-01,Central,NaN,NaN,Central,Single Family Residential,NaN,NaN


### Show df info()

In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 52319 entries, 0 to 52318
Data columns (total 69 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   typeconstructiontypeid        76 non-null     float64
 1   storytypeid                   47 non-null     float64
 2   propertylandusetypeid         52319 non-null  float64
 3   heatingorsystemtypeid         33849 non-null  float64
 4   buildingclasstypeid           0 non-null      float64
 5   architecturalstyletypeid      70 non-null     float64
 6   airconditioningtypeid         13615 non-null  float64
 7   parcelid                      52319 non-null  int64  
 8   id                            52319 non-null  int64  
 9   basementsqft                  47 non-null     float64
 10  bathroomcnt                   52319 non-null  float64
 11  bedroomcnt                    52319 non-null  float64
 12  buildingqualitytypeid         33654 non-null  float64
 13  c

In [12]:
# df shape
df.shape

(52319, 69)

In [13]:
# Show duplicated observations even with SQL filter
df.parcelid.duplicated().sum()

0

In [14]:
# Sort df by parcelid and then by transaction date to group by parcelid and to make sure transaction date is last
df = df.sort_values(by=['parcelid','transactiondate'])
df.head()

,typeconstructiontypeid,storytypeid,propertylandusetypeid,heatingorsystemtypeid,buildingclasstypeid,architecturalstyletypeid,airconditioningtypeid,parcelid,id,basementsqft,...,id.1,logerror,transactiondate,airconditioningdesc,architecturalstyledesc,buildingclassdesc,heatingorsystemdesc,propertylandusedesc,storydesc,typeconstructiondesc
37107,NaN,NaN,261.0,2.0,NaN,NaN,NaN,10711855,1087254,NaN,...,55006,-0.007357,2017-07-07,NaN,NaN,NaN,Central,Single Family Residential,NaN,NaN
48127,NaN,NaN,261.0,2.0,NaN,NaN,1.0,10711877,1072280,NaN,...,71382,0.021066,2017-08-29,Central,NaN,NaN,Central,Single Family Residential,NaN,NaN
15458,NaN,NaN,261.0,2.0,NaN,NaN,1.0,10711888,1340933,NaN,...,23209,0.077174,2017-04-04,Central,NaN,NaN,Central,Single Family Residential,NaN,NaN
12040,NaN,NaN,261.0,2.0,NaN,NaN,NaN,10711910,1878109,NaN,...,18017,-0.041238,2017-03-17,NaN,NaN,NaN,Central,Single Family Residential,NaN,NaN
13593,NaN,NaN,261.0,2.0,NaN,NaN,NaN,10711923,2190858,NaN,...,20378,-0.009496,2017-03-24,NaN,NaN,NaN,Central,Single Family Residential,NaN,NaN


In [15]:
# Show duplicated parcel id
df[df.duplicated(subset='parcelid', keep=False)].head()

,typeconstructiontypeid,storytypeid,propertylandusetypeid,heatingorsystemtypeid,buildingclasstypeid,architecturalstyletypeid,airconditioningtypeid,parcelid,id,basementsqft,...,id.1,logerror,transactiondate,airconditioningdesc,architecturalstyledesc,buildingclassdesc,heatingorsystemdesc,propertylandusedesc,storydesc,typeconstructiondesc


In [16]:
# drop dulicates  but keep last transaction date
df = df.drop_duplicates(subset='parcelid', keep='last')

In [17]:
# Show duplicated parcel id
df[df.duplicated(subset='parcelid', keep=False)].head()

,typeconstructiontypeid,storytypeid,propertylandusetypeid,heatingorsystemtypeid,buildingclasstypeid,architecturalstyletypeid,airconditioningtypeid,parcelid,id,basementsqft,...,id.1,logerror,transactiondate,airconditioningdesc,architecturalstyledesc,buildingclassdesc,heatingorsystemdesc,propertylandusedesc,storydesc,typeconstructiondesc


In [18]:
# confirm duplicated parcel id removed
df.parcelid.duplicated().sum()

0

In [19]:
# check new shape
df.shape

(52319, 69)

In [20]:
# Replace blank values with NaN
df = df.replace('',np.nan)

In [21]:
# check if long/lat columns have nulls
df.longitude.isnull().sum(), df.latitude.isnull().sum()

(0, 0)

In [22]:
# number of nulls per column
df.isna().sum()

typeconstructiontypeid    52243
storytypeid               52272
propertylandusetypeid         0
heatingorsystemtypeid     18470
buildingclasstypeid       52319
                          ...  
buildingclassdesc         52319
heatingorsystemdesc       18470
propertylandusedesc           0
storydesc                 52272
typeconstructiondesc      52243
Length: 69, dtype: int64

In [23]:
# percent of nulls per column
df.isna().mean()

typeconstructiontypeid    0.998547
storytypeid               0.999102
propertylandusetypeid     0.000000
heatingorsystemtypeid     0.353027
buildingclasstypeid       1.000000
                            ...   
buildingclassdesc         1.000000
heatingorsystemdesc       0.353027
propertylandusedesc       0.000000
storydesc                 0.999102
typeconstructiondesc      0.998547
Length: 69, dtype: float64

In [24]:
# number of nulls per row
df.isna().sum(axis=1)

37107    31
48127    29
15458    31
12040    31
13593    31
         ..
42188    35
31037    36
23810    36
8954     48
28006    48
Length: 52319, dtype: int64

In [25]:
# number of rows with a specific number of nulls
df.isna().sum(axis=1).value_counts()

33    11967
34    11157
32     8885
31     5989
36     4138
35     3469
29     2527
30     2199
37     1020
28      390
38      223
27      177
44       46
26       30
39       29
40       15
24       12
25       11
43       10
42        7
45        6
41        6
23        2
48        2
46        1
47        1
dtype: int64

In [26]:
# create dataframe that has column name as first column
col_nulls = pd.DataFrame()
col_nulls['columns_name'] = df.isna().sum().index
col_nulls

,columns_name
0,typeconstructiontypeid
1,storytypeid
2,propertylandusetypeid
3,heatingorsystemtypeid
4,buildingclasstypeid
...,...
64,buildingclassdesc
65,heatingorsystemdesc
66,propertylandusedesc
67,storydesc


In [27]:
# create new column that hold the sum of nulls from each column
col_nulls['row_null_count'] = df.isna().sum().values
col_nulls

,columns_name,row_null_count
0,typeconstructiontypeid,52243
1,storytypeid,52272
2,propertylandusetypeid,0
3,heatingorsystemtypeid,18470
4,buildingclasstypeid,52319
...,...,...
64,buildingclassdesc,52319
65,heatingorsystemdesc,18470
66,propertylandusedesc,0
67,storydesc,52272


In [28]:
# create new column that hold the average of nulls from each column
col_nulls['row_null_percent'] = df.isna().mean().values
col_nulls

,columns_name,row_null_count,row_null_percent
0,typeconstructiontypeid,52243,0.998547
1,storytypeid,52272,0.999102
2,propertylandusetypeid,0,0.000000
3,heatingorsystemtypeid,18470,0.353027
4,buildingclasstypeid,52319,1.000000
...,...,...,...
64,buildingclassdesc,52319,1.000000
65,heatingorsystemdesc,18470,0.353027
66,propertylandusedesc,0,0.000000
67,storydesc,52272,0.999102


In [29]:
# sort values by percent
col_nulls = col_nulls.sort_values(by=['row_null_percent'], ascending=False)
col_nulls

,columns_name,row_null_count,row_null_percent
18,finishedsquarefeet13,52319,1.000000
64,buildingclassdesc,52319,1.000000
4,buildingclasstypeid,52319,1.000000
19,finishedsquarefeet15,52319,1.000000
67,storydesc,52272,0.999102
...,...,...,...
28,latitude,0,0.000000
29,longitude,0,0.000000
43,roomcnt,0,0.000000
38,rawcensustractandblock,0,0.000000


In [30]:
def col_nulls(df):
    # create dataframe that has column name as first column
    col_nulls = pd.DataFrame()
    col_nulls['columns_name'] = df.isna().sum().index

    # create new column that hold the sum of nulls from each column
    col_nulls['row_null_count'] = df.isna().sum().values


    # create new column that hold the average of nulls from each column
    col_nulls['row_null_percent'] = df.isna().mean().values


    # sort values by percent
    col_nulls = col_nulls.sort_values(by=['row_null_percent'], ascending=False)
    
    return col_nulls

In [31]:
col_nulls = col_nulls(df)
col_nulls

,columns_name,row_null_count,row_null_percent
18,finishedsquarefeet13,52319,1.000000
64,buildingclassdesc,52319,1.000000
4,buildingclasstypeid,52319,1.000000
19,finishedsquarefeet15,52319,1.000000
67,storydesc,52272,0.999102
...,...,...,...
28,latitude,0,0.000000
29,longitude,0,0.000000
43,roomcnt,0,0.000000
38,rawcensustractandblock,0,0.000000


In [32]:
# Create df with number of rows with a specific number of null columns
row_nulls = pd.DataFrame(df.isna().sum(axis=1).value_counts(), columns=['num_rows_with_n_null_cols'])
row_nulls.head()

,num_rows_with_n_null_cols
33,11967
34,11157
32,8885
31,5989
36,4138


In [33]:
# make first columnb the number of nulls
row_nulls = row_nulls.reset_index()
row_nulls.head()

,index,num_rows_with_n_null_cols
0,33,11967
1,34,11157
2,32,8885
3,31,5989
4,36,4138


In [34]:
# rename index to match values
row_nulls = row_nulls.rename(columns={'index':'n_null_cols'})
row_nulls.head()

,n_null_cols,num_rows_with_n_null_cols
0,33,11967
1,34,11157
2,32,8885
3,31,5989
4,36,4138


In [35]:
# create columsn for percent of null cols
row_nulls['percent_null_cols'] = row_nulls.n_null_cols / df.shape[1]
row_nulls.head()

,n_null_cols,num_rows_with_n_null_cols,percent_null_cols
0,33,11967,0.478261
1,34,11157,0.492754
2,32,8885,0.463768
3,31,5989,0.449275
4,36,4138,0.521739


In [36]:
# sort df by percentn of null cols
row_nulls = row_nulls.sort_values(by=['percent_null_cols'], ascending=False)
row_nulls

,n_null_cols,num_rows_with_n_null_cols,percent_null_cols
23,48,2,0.695652
25,47,1,0.681159
24,46,1,0.666667
20,45,6,0.652174
12,44,46,0.637681
18,43,10,0.623188
19,42,7,0.608696
21,41,6,0.594203
15,40,15,0.579710
14,39,29,0.565217


In [37]:
def row_nulls(df):
    # Create df with number of rows with a specific number of null columns
    row_nulls = pd.DataFrame(df.isna().sum(axis=1).value_counts(), columns=['num_rows_with_n_null_cols'])

    # make first columnb the number of nulls
    row_nulls = row_nulls.reset_index()

    # rename index to match values
    row_nulls = row_nulls.rename(columns={'index':'n_null_cols'})

    # create columsn for percent of null cols
    row_nulls['percent_null_cols'] = row_nulls.n_null_cols / df.shape[1]

    # sort df by percentn of null cols
    row_nulls = row_nulls.sort_values(by=['percent_null_cols'], ascending=False)
    
    return row_nulls



In [38]:
row_nulls = row_nulls(df)
row_nulls

,n_null_cols,num_rows_with_n_null_cols,percent_null_cols
23,48,2,0.695652
25,47,1,0.681159
24,46,1,0.666667
20,45,6,0.652174
12,44,46,0.637681
18,43,10,0.623188
19,42,7,0.608696
21,41,6,0.594203
15,40,15,0.579710
14,39,29,0.565217


# <font color = 'red'> Clean

In [39]:
# show value counts for each column, does not include nulls
for col in df.columns:
    print(f'-------------{col}-----------------')
    print(df[df[col].notna()][col].value_counts(dropna=False))

-------------typeconstructiontypeid-----------------
6.0    75
4.0     1
Name: typeconstructiontypeid, dtype: int64
-------------storytypeid-----------------
7.0    47
Name: storytypeid, dtype: int64
-------------propertylandusetypeid-----------------
261.0    52319
Name: propertylandusetypeid, dtype: int64
-------------heatingorsystemtypeid-----------------
2.0     20689
7.0     12526
6.0       517
20.0       85
13.0       16
1.0         7
18.0        6
10.0        2
24.0        1
Name: heatingorsystemtypeid, dtype: int64
-------------buildingclasstypeid-----------------
Series([], Name: buildingclasstypeid, dtype: int64)
-------------architecturalstyletypeid-----------------
7.0     62
3.0      3
2.0      2
21.0     2
8.0      1
Name: architecturalstyletypeid, dtype: int64
-------------airconditioningtypeid-----------------
1.0     11873
13.0     1567
5.0       159
11.0       16
Name: airconditioningtypeid, dtype: int64
-------------parcelid-----------------
10913809    1
14004381   

15.0    1124
14.0     559
13.0     126
12.0      81
11.0      79
10.0      47
9.0       33
8.0        8
7.0        8
6.0        1
99.0       1
4.0        1
Name: taxdelinquencyyear, dtype: int64
-------------censustractandblock-----------------
6.037920e+13    32
6.037920e+13    27
6.037142e+13    24
6.059032e+13    24
6.059032e+13    23
                ..
6.037601e+13     1
6.037141e+13     1
6.111000e+13     1
6.037702e+13     1
6.037134e+13     1
Name: censustractandblock, Length: 31404, dtype: int64
-------------id.1-----------------
0        1
48573    1
3499     1
13740    1
11695    1
        ..
72310    1
70263    1
43648    1
41601    1
2047     1
Name: id.1, Length: 52319, dtype: int64
-------------logerror-----------------
 0.000231    5
 0.000727    5
 0.001878    5
 0.002652    5
 0.008257    4
            ..
 0.035055    1
-0.153667    1
 0.087675    1
-0.005029    1
 0.025578    1
Name: logerror, Length: 52060, dtype: int64
-------------transactiondate-----------------
2

In [40]:
df.head()

,typeconstructiontypeid,storytypeid,propertylandusetypeid,heatingorsystemtypeid,buildingclasstypeid,architecturalstyletypeid,airconditioningtypeid,parcelid,id,basementsqft,...,id.1,logerror,transactiondate,airconditioningdesc,architecturalstyledesc,buildingclassdesc,heatingorsystemdesc,propertylandusedesc,storydesc,typeconstructiondesc
37107,NaN,NaN,261.0,2.0,NaN,NaN,NaN,10711855,1087254,NaN,...,55006,-0.007357,2017-07-07,NaN,NaN,NaN,Central,Single Family Residential,NaN,NaN
48127,NaN,NaN,261.0,2.0,NaN,NaN,1.0,10711877,1072280,NaN,...,71382,0.021066,2017-08-29,Central,NaN,NaN,Central,Single Family Residential,NaN,NaN
15458,NaN,NaN,261.0,2.0,NaN,NaN,1.0,10711888,1340933,NaN,...,23209,0.077174,2017-04-04,Central,NaN,NaN,Central,Single Family Residential,NaN,NaN
12040,NaN,NaN,261.0,2.0,NaN,NaN,NaN,10711910,1878109,NaN,...,18017,-0.041238,2017-03-17,NaN,NaN,NaN,Central,Single Family Residential,NaN,NaN
13593,NaN,NaN,261.0,2.0,NaN,NaN,NaN,10711923,2190858,NaN,...,20378,-0.009496,2017-03-24,NaN,NaN,NaN,Central,Single Family Residential,NaN,NaN


In [41]:
df.shape

(52319, 69)

In [42]:
# remove propeties that have no bedrooms and no bathrooms and too small of an area
df = df[(df.bedroomcnt > 0) & (df.bathroomcnt > 0) & (df.unitcnt <= 1) | df.unitcnt.isna() & (df.calculatedfinishedsquarefeet > 500) & (df.bedroomcnt > 0) & (df.bathroomcnt > 0)]

In [43]:
df.shape

(52114, 69)

In [45]:
dft = df

In [90]:
df = dft

In [67]:
df.shape

(52114, 69)

In [68]:
def handle_nulls(df, percent_required_cols = .5, percent_required_rows = .7):
    
    # set threshold for min of values in columns for dropping
    thresh_col = int(round(percent_required_cols * df.shape[0]))
    
    # drop columns that don't meed threshhold for non-null values (rows without nulls)
    df = df.dropna(axis=1, thresh=thresh_col)
    
    # set threshold for min non-null values for rows (cols without nulls)
    thresh_row = int(round(percent_required_rows * df.shape[1]))
    
    # drop rows with don't meet threshold for non-null values for columns
    df = df.dropna(axis=0, thresh=thresh_row)
    
    return df

In [69]:
# Run fucntion to remove columns and rows that meet thresh hold
df, thresh_col, thresh_row  = handle_nulls(df, percent_required_cols = .5, percent_required_rows = .7)

In [70]:
# see new df shape, 
df.shape, thresh_col, thresh_row

((52114, 35), 26057, 24)

### Deal with the rest of the nulls

In [ ]:
df.isna().sum()

### Dropping Unneeded Columns:
- parcel id, id, popertylandusetype, unitcnt: no longer needed after data filtering
- propertyzoningdesc, finishedsquarefeet12, censustracdtandblock, rawcensustracdtandblock, heatingorsystemtupeid, assessmentyear, transactiondate: Not useful

In [71]:
# remove columns that are not useful
df = df.drop(columns=[
    # uniquie identifer to lot
    'parcelid',
    # uniquie identifer for table        
    'id',
     #Description of the allowed land uses (zoning) for that property
     'propertyzoningdesc', 
     # Finished living area
     'finishedsquarefeet12',
     #  Census tract and block ID combined - also contains blockgroup assignment by extension
         'censustractandblock',
      # Type of land use the property is zoned for
      'propertylandusetypeid',
      #  Type of home heating system
      'heatingorsystemtypeid',
    # unit type cnt: filtered out to only 1 cnt, no longer needed
    'unitcnt',
    # Census tract and block ID combined, not needed
    'rawcensustractandblock',
    # year assessed
    'assessmentyear',
    # date of transaction
    'transactiondate',
    #  Number of bathrooms in home including fractional bathroom. duplicate from bathroomcnt
    'calculatedbathnbr',
    #  Total number of rooms in the principal residence. Not collected for LA County
    'roomcnt',
    # descirpiton of land use (single family), no longer needed
    'propertylandusedesc',
    # duplicate id column
    'id.1'
    
    ])

In [72]:
df.shape

(52114, 20)

In [ ]:
# check value counts for heating systems
df.heatingorsystemdesc.value_counts(dropna=False)

In [ ]:
# check value counts for heating sytems in LA
df[df.fips == 6037].heatingorsystemdesc.value_counts(dropna=False)

In [ ]:
# check value counts for heating sytems in OC
df[df.fips == 6059].heatingorsystemdesc.value_counts(dropna=False)

In [ ]:
# check value counts for heating sytems in Ventura
df[df.fips == 6111].heatingorsystemdesc.value_counts(dropna=False)

In [73]:
# relacing nulls with 'None', assuming null was for not having a heating system
df.heatingorsystemdesc.fillna('None', inplace=True)

In [74]:
df.shape

(52114, 20)

In [ ]:
# checking nulls were removed
df.heatingorsystemdesc.isna().sum()

In [ ]:
# checking building quality type id for LA
df[df.fips == 6037].buildingqualitytypeid.value_counts(dropna=False)

In [ ]:
# checking building quality type id for OC
df[df.fips == 6059].buildingqualitytypeid.value_counts(dropna=False)

In [ ]:
# checking building quality type id for Vetura
df[df.fips == 6111].buildingqualitytypeid.value_counts(dropna=False)

### Looks like building quality type was not collected for OC or Ventura. Dropping column.

In [75]:
# droping buildingqualitytypeid because they are not collected for Ventura and Orange
df.drop(columns=['buildingqualitytypeid'], inplace=True)

In [76]:
df.shape

(52114, 19)

In [ ]:
df.isna().sum()

In [77]:
# dropping the rest of the nulls
df = df.dropna()

In [78]:
df.isna().sum()

bathroomcnt                     0
bedroomcnt                      0
calculatedfinishedsquarefeet    0
fips                            0
fullbathcnt                     0
latitude                        0
longitude                       0
lotsizesquarefeet               0
propertycountylandusecode       0
regionidcity                    0
regionidcounty                  0
regionidzip                     0
yearbuilt                       0
structuretaxvaluedollarcnt      0
taxvaluedollarcnt               0
landtaxvaluedollarcnt           0
taxamount                       0
logerror                        0
heatingorsystemdesc             0
dtype: int64

In [79]:
df.shape

(50676, 19)

In [87]:
df = dft

In [91]:
def handle_nulls(df, percent_required_cols = .5, percent_required_rows = .7):
    
    # set threshold for min of values in columns for dropping
    thresh_col = int(round(percent_required_cols * df.shape[0]))
    
    # drop columns that don't meed threshhold for non-null values (rows without nulls)
    df = df.dropna(axis=1, thresh=thresh_col)
    
    # set threshold for min non-null values for rows (cols without nulls)
    thresh_row = int(round(percent_required_rows * df.shape[1]))
    
    # drop rows with don't meet threshold for non-null values for columns
    df = df.dropna(axis=0, thresh=thresh_row)
    
    # remove columns that are not useful
    df = df.drop(columns=[
        # uniquie identifer to lot
        'parcelid',
        # uniquie identifer for table        
        'id',
         #Description of the allowed land uses (zoning) for that property
         'propertyzoningdesc', 
         # Finished living area
         'finishedsquarefeet12',
         #  Census tract and block ID combined - also contains blockgroup assignment by extension
             'censustractandblock',
          # Type of land use the property is zoned for
          'propertylandusetypeid',
          #  Type of home heating system
          'heatingorsystemtypeid',
        # unit type cnt: filtered out to only 1 cnt, no longer needed
        'unitcnt',
        # Census tract and block ID combined, not needed
        'rawcensustractandblock',
        # year assessed
        'assessmentyear',
        # date of transaction
        'transactiondate',
        #  Number of bathrooms in home including fractional bathroom. duplicate from bathroomcnt
        'calculatedbathnbr',
        #  Total number of rooms in the principal residence. Not collected for LA County
        'roomcnt',
        # descirpiton of land use (single family), no longer needed
        'propertylandusedesc',
        # duplicate id column
        'id.1'

        ])
    
    # relacing nulls with 'None', assuming null was for not having a heating system
    df.heatingorsystemdesc.fillna('None', inplace=True)
    
    # droping buildingqualitytypeid because they are not collected for Ventura and Orange
    df.drop(columns=['buildingqualitytypeid'], inplace=True)
    
    # dropping the rest of the nulls
    df = df.dropna()
    
    return df

In [92]:
df = handle_nulls(df, percent_required_cols = .5, percent_required_rows = .7)
df.shape

(50676, 19)

## Remove Outliers

In [ ]:
# List of columns
cols = [col for col in df.columns.drop(['bedroomcnt',
                                        'fips',
                                        'propertycountylandusecode',
                                        'heatingorsystemdesc'
                                       ])]
cols

In [ ]:
plt.figure(figsize=(15, 5))

for i, col in enumerate(cols,1):

    # i starts at 0, but plot nos should start at 1
    plot_number = i

    # Create subplot.
    plt.subplot(1, len(cols), plot_number)

    # Title with column name.
    plt.title(col)

    # Display histogram for column.
    df[col].hist()

    # Hide gridlines.
    plt.grid()
    
    # turn off scientific notation
    #plt.ticklabel_format(useOffset=False)
    
plt.show()

In [ ]:
df.shape

In [ ]:
# List of columns
        
plt.figure(figsize=(16, 20))

for i, col in enumerate(cols):

    # i starts at 0, but plot nos should start at 1
    plot_number = i + 1

    # Create subplot.
    plt.subplot(1, len(cols), plot_number)

    # Title with column name.
    plt.title(col)

    # Display boxplot for column.
    sns.boxplot(data=df[[col]])

    # Hide gridlines.
    plt.grid(False)
    
    # sets proper spacing between plots
    plt.tight_layout()

plt.show()

In [ ]:
# remove outliers from each column in cols_list
for col in cols:

    q1, q3 = df[col].quantile([.25, .75])  # get quartiles

    iqr = q3 - q1   # calculate interquartile range

    upper_bound = q3 + 1.5 * iqr   # get upper bound
    lower_bound = q1 - 1.5 * iqr   # get lower bound

    # return dataframe without outliers

    df = df[(df[col] > lower_bound) & (df[col] < upper_bound)]

In [ ]:
df.shape

In [ ]:
plt.figure(figsize=(15, 5))

for i, col in enumerate(cols,1):

    # i starts at 0, but plot nos should start at 1
    plot_number = i

    # Create subplot.
    plt.subplot(1, len(cols), plot_number)

    # Title with column name.
    plt.title(col)

    # Display histogram for column.
    df[col].hist()

    # Hide gridlines.
    plt.grid()
    
    # turn off scientific notation
    #plt.ticklabel_format(useOffset=False)
    
plt.show()

In [ ]:
# List of columns
        
plt.figure(figsize=(16, 20))

for i, col in enumerate(cols):

    # i starts at 0, but plot nos should start at 1
    plot_number = i + 1

    # Create subplot.
    plt.subplot(1, len(cols), plot_number)

    # Title with column name.
    plt.title(col)

    # Display boxplot for column.
    sns.boxplot(data=df[[col]])

    # Hide gridlines.
    plt.grid(False)
    
    # sets proper spacing between plots
    plt.tight_layout()

plt.show()

## Rename for Readability

In [ ]:
df.columns.to_list()

In [ ]:
# rename for readability
df = df.rename(columns = {'bathroomcnt':'bathrooms',
 'bedroomcnt':'bedrooms',
 'calculatedfinishedsquarefeet':'house_area',
 'fullbathcnt':'full_baths',
 'lotsizesquarefeet':'lot_area',
 'propertycountylandusecode':'land_use_code',
 'regionidcity':'city_id',
 'regionidcounty':'county_id',
 'regionidzip':'zip_id',
 'yearbuilt':'year_built',
 'structuretaxvaluedollarcnt':'tax_value_house',
 'taxvaluedollarcnt':'tax_value_total',
 'landtaxvaluedollarcnt':'tax_value_land',
 'taxamount':'tax_amount',
 'heatingorsystemdesc':'heating_sys',
})

In [ ]:
df.columns.to_list()

## Add Columns: Age, Absolute LogError, County, HouseDolloarPerSqft, TaxRate, BedBath Ratio

In [ ]:
# create column for age of house
df['age'] = 2017 - df.year_built
df.head()

In [ ]:
# Create column for absolute value of logerror
df['logerror_abs'] = df.logerror.abs()
df.head()

In [ ]:
# create columsn to match fips to county(LA, Orange County, Ventury County)
df['county'] = df.fips.map({6037:'LA',
            6059:'OC',
            6111:'Ventura'})
df.head()

In [ ]:
# create column for Dollar per square foot
df['house_dollar_sqft'] = df.tax_value_total / df.house_area
df.head()

In [ ]:
# Create column for land dollar per sqft
df['land_dollar_sqft'] = df.tax_value_land / df.lot_area
df.head()

In [ ]:
# create column for tax rate
df['tax_rate'] = df.tax_value_house / df.tax_amount
df.head()

In [ ]:
# create column for bed bath ratio
df['bed_bath_ratio'] = df.bedrooms / df.bathrooms
df.head()

In [ ]:
df.columns.to_list()

In [94]:
def fix_cols(df):

    # rename for readability
    df = df.rename(columns = {'bathroomcnt':'bathrooms',
     'bedroomcnt':'bedrooms',
     'calculatedfinishedsquarefeet':'house_area',
     'fullbathcnt':'full_baths',
     'lotsizesquarefeet':'lot_area',
     'propertycountylandusecode':'land_use_code',
     'regionidcity':'city_id',
     'regionidcounty':'county_id',
     'regionidzip':'zip_id',
     'yearbuilt':'year_built',
     'structuretaxvaluedollarcnt':'tax_value_house',
     'taxvaluedollarcnt':'tax_value_total',
     'landtaxvaluedollarcnt':'tax_value_land',
     'taxamount':'tax_amount',
     'heatingorsystemdesc':'heating_sys',
    })



    # create column for age of house
    df['age'] = 2017 - df.year_built


    # Create column for absolute value of logerror
    df['logerror_abs'] = df.logerror.abs()


    # create columsn to match fips to county(LA, Orange County, Ventury County)
    df['county'] = df.fips.map({6037:'LA',
                6059:'OC',
                6111:'Ventura'})


    # create column for Dollar per square foot
    df['house_dollar_sqft'] = df.tax_value_total / df.house_area


    # Create column for land dollar per sqft
    df['land_dollar_sqft'] = df.tax_value_land / df.lot_area


    # create column for tax rate
    df['tax_rate'] = df.tax_value_house / df.tax_amount


    # create column for bed bath ratio
    df['bed_bath_ratio'] = df.bedrooms / df.bathrooms
    
    return df

In [95]:
df = fix_cols(df)
df.shape

(50676, 26)

In [96]:
df.columns

Index(['bathrooms', 'bedrooms', 'house_area', 'fips', 'full_baths', 'latitude',
       'longitude', 'lot_area', 'land_use_code', 'city_id', 'county_id',
       'zip_id', 'year_built', 'tax_value_house', 'tax_value_total',
       'tax_value_land', 'tax_amount', 'logerror', 'heating_sys', 'age',
       'logerror_abs', 'county', 'house_dollar_sqft', 'land_dollar_sqft',
       'tax_rate', 'bed_bath_ratio'],
      dtype='object')

# <font color = 'red'>Split

In [ ]:
train_validate, test = train_test_split(df, test_size=.2, random_state=123)
train, validate = train_test_split(train_validate, test_size=.3, random_state=123)
train.shape, validate.shape, test.shape

In [100]:
def split(df):
    train_validate, test = train_test_split(df, test_size=.2, random_state=123)
    train, validate = train_test_split(train_validate, test_size=.3, random_state=123)
    return train, validate, test

In [102]:
train, validate, test = split(df)
train.shape, validate.shape, test.shape

((28378, 26), (12162, 26), (10136, 26))

# <font color = 'red'> Explore Unscaled Data

In [ ]:
sns.relplot(data=train, x='age', y='house_dollar_sqft', hue='logerror_abs', col='county', kind='scatter')

In [ ]:
# plot without county
sns.relplot(data=train, x='age', y='house_dollar_sqft', hue='logerror_abs', kind='scatter')

In [ ]:
# show log error for age vs land dollar per sqft
sns.relplot(data=train, x='age', y='land_dollar_sqft', hue='logerror_abs', kind='scatter')

In [ ]:
# show log error for age vs land dollar per sqft
sns.relplot(data=train, x='land_dollar_sqft', y='house_dollar_sqft', hue='logerror_abs', kind='scatter')

### Takeaways:
- ???

### Viz...

### Note:
- high log error cluster with high age and low house area
- high log error cluster with high age and low tax value
- hig log error cluster with high lot area and middle age

### Go Forward with Clustering Combos:
- age
- house dollar per sqft
- lot dollar per dollar per 
- tax_value
- longitude
- latitude

# <font color = 'red'> Scale Data

In [ ]:
# select columns to cluster and explore
cols = ['house_area', 'lot_area', 'tax_value_house', 'tax_value_total',
       'tax_value_land', 'age',
       'logerror_abs','house_dollar_sqft', 'land_dollar_sqft',
       'tax_rate', 'bed_bath_ratio', 'fips']

In [ ]:
# Columns used for further exploration and clustering
train_cols = train[cols]
validate_cols = validate[cols]
test_cols = test[cols]
train_cols.head()

In [ ]:
# Make the scaler with MinMax
scaler = MinMaxScaler()

# Fit the scalter to X_train
scaler.fit(train_cols)

# Transform train, validate, test to scaled version
train_scaled = scaler.transform(train_cols)
validate_scaled = scaler.transform(validate_cols)
test_scaled = scaler.transform(test_cols)

In [ ]:
# Make the scaled arrays into dataframes
train_scaled = pd.DataFrame(train_scaled, columns=train_cols.columns, index=train_cols.index)
validate_scaled = pd.DataFrame(validate_scaled, columns=validate_cols.columns, index=validate_cols.index)
test_scaled = pd.DataFrame(test_scaled, columns=test_cols.columns, index=test_cols.index)
train_scaled

In [103]:
def scale_cols(train, validate, test):
    
    # Scaler
    from sklearn.preprocessing import MinMaxScaler
    
    # select columns to cluster and explore
    cols = ['house_area', 'lot_area', 'tax_value_house', 'tax_value_total',
           'tax_value_land', 'age',
           'logerror_abs','house_dollar_sqft', 'land_dollar_sqft',
           'tax_rate', 'bed_bath_ratio', 'fips']

    # Columns used for further exploration and clustering
    train_cols = train[cols]
    validate_cols = validate[cols]
    test_cols = test[cols]

    # Make the scaler with MinMax
    scaler = MinMaxScaler()

    # Fit the scalter to X_train
    scaler.fit(train_cols)

    # Transform train, validate, test to scaled version
    train_scaled = scaler.transform(train_cols)
    validate_scaled = scaler.transform(validate_cols)
    test_scaled = scaler.transform(test_cols)

    # Make the scaled arrays into dataframes
    train_scaled = pd.DataFrame(train_scaled, columns=train_cols.columns, index=train_cols.index)
    validate_scaled = pd.DataFrame(validate_scaled, columns=validate_cols.columns, index=validate_cols.index)
    test_scaled = pd.DataFrame(test_scaled, columns=test_cols.columns, index=test_cols.index)
    
    return train_scaled, validate_scaled, test_scaled

In [104]:
train_scaled, validate_scaled, test_scaled = scale_cols(train, validate, test)
train_scaled.shape, validate_scaled.shape, test_scaled.shape

((28378, 12), (12162, 12), (10136, 12))

## <font color = 'red'>Explore Scaled Data

### Viz Age for Land Dollar per Sqft vs House Dollar per Sqft by FIP

In [ ]:
sns.relplot(data=train_scaled, x='land_dollar_sqft', y='house_dollar_sqft', hue='age', col='fips', kind='scatter')

### Takeaways:
- Age works for clustering

### Viz Log Error for Age for House Dollar Sqft per Fip

In [ ]:
sns.relplot(data=train_scaled, x='age', y='house_dollar_sqft', hue='logerror_abs', col='fips',kind='scatter')

### Takeaways:
- Logerror clusters with older houses

### Viz Bed Bath Ratio for  Age for House Dollar Sqft per Fip

In [ ]:
sns.relplot(data=train_scaled, x='age', y='house_dollar_sqft', hue='bed_bath_ratio', col='fips', kind='scatter')

### Takeaways:
- bed bath ratio clusters wwith older homes

### Viz Tax Rate for  Age vs House Dollar Sqft per Fip

In [ ]:
sns.relplot(data=train_scaled, x='age', y='house_dollar_sqft', hue='tax_rate', col='fips', kind='scatter')

### Takeaways:
- Tax Rate clusters with older homes

## Forward with Age, Tax Rate, Bed_Bath_Ratio, House_Dollar_Sqft, Land_Dollar_Sqft
- Best K = 3 (Age, Tax Rate, Bed_Bath_Ratio)

# <font color = 'red'> Clustering Combo 1 (House Dollar Per Sqft, Age)

In [ ]:
# Filter columns from train scaled
cols = ['house_dollar_sqft', 'age']
train_scaled_X = train_scaled[cols]
train_scaled_X.head()

In [ ]:
sns.relplot(data=train_scaled, x='age', y='house_dollar_sqft', hue= train.logerror_abs, kind='scatter')

## <font color = 'red'>Elbow Method

In [ ]:
# Use elbow method to see if inertia values support visual exploration
# plot inertia vs k

with plt.style.context('seaborn-whitegrid'):
    plt.figure(figsize=(9, 6))
    pd.Series({k: KMeans(k, random_state=123).fit(train_scaled).inertia_ for k in range(2, 12)}).plot(marker='x')
    plt.xticks(range(2, 12))
    plt.xlabel('k')
    plt.ylabel('inertia')
    plt.title('Change in inertia as k increases')

### Takeaway:
- K vallues between 3 - 5
- Forward with k = 3

In [ ]:
# Move forward with k=3
kmeans = KMeans(n_clusters = 3, random_state=123)
kmeans.fit(train_scaled_X)

# And assign the cluster number to a column on the dataframe
train["cluster"] = kmeans.predict(train_scaled_X)
train.head()

In [ ]:
# show house dollar per sqft for age vs log error by cluster
sns.relplot(data=train, x="age", y="house_dollar_sqft", col="cluster", hue='logerror_abs', col_wrap=2)

### Takeaways:
- cluster 2 shows highest log errors

### Check Significant Difference

In [ ]:
# check mean of logerror for each cluster
train.groupby(by='cluster').logerror_abs.mean()

### Takeaways:
- Mean for cluster two is greater than the rest

### Run T-Test
- H0: Cluster 2 mean <= Overall Mean
- HA: Cluster 2 mean > Overall Mean 

In [ ]:
# run one sample T-Test
alpha = 0.05
cluster_logerror = train[train.cluster==2].logerror_abs
overall_logerror = train.logerror_abs.mean()

t, p = stats.ttest_1samp(cluster_logerror, overall_logerror)
print(f't={t}, p={p/2}, alpha={alpha}')

### Takeaways:
- Since T > 0 and p < alpah, then reject Null Hypothesis. Cluster 2 Logerror is Significantly Greater than others

### ???

In [105]:
def cluster_1(train_scaled, train):
    # cluster method
    from sklearn.cluster import KMeans
    
    # Filter columns from train scaled
    cols = ['house_dollar_sqft', 'age']
    train_scaled_X = train_scaled[cols]

    # Move forward with k=3
    kmeans = KMeans(n_clusters = 3, random_state=123)
    kmeans.fit(train_scaled_X)

    # And assign the cluster number to a column on the dataframe
    train["cluster"] = kmeans.predict(train_scaled_X)

    # run one sample T-Test
    alpha = 0.05
    cluster_logerror = train[train.cluster==2].logerror_abs
    overall_logerror = train.logerror_abs.mean()

    t, p = stats.ttest_1samp(cluster_logerror, overall_logerror)
    
    return t, p, train

In [106]:
t, p, train = cluster_1(train_scaled, train)
t, p, train.shape

(-0.9739375596758768, 0.3301027008270979, (28378, 27))

# <font color = 'red'> Clustering Combo 2 (House Dollar Per Sqft, Age, Bed Bath Ratio)

In [ ]:
# Filter columsn from train scaled
cols = ['age', 'house_dollar_sqft', 'bed_bath_ratio']
train_scaled_X = train_scaled[cols]
train_scaled_X.head()

In [ ]:
sns.relplot(data=train_scaled, x='age', y='house_dollar_sqft', hue= train.logerror_abs, kind='scatter', height=6, aspect=2)

In [ ]:
# Use elbow method to see if inertia values support visual exploration
# plot inertia vs k

with plt.style.context('seaborn-whitegrid'):
    plt.figure(figsize=(9, 6))
    pd.Series({k: KMeans(k, random_state=123).fit(train_scaled).inertia_ for k in range(2, 12)}).plot(marker='x')
    plt.xticks(range(2, 12))
    plt.xlabel('k')
    plt.ylabel('inertia')
    plt.title('Change in inertia as k increases')

In [ ]:
# Move forward with k=3
kmeans = KMeans(n_clusters = 4, random_state=123)
kmeans.fit(train_scaled_X)

# And assign the cluster number to a column on the dataframe
train["cluster"] = kmeans.predict(train_scaled_X)
train.head()

In [ ]:
# show house dollar per sqft for age vs log error by cluster
sns.relplot(data=train, x="age", y="house_dollar_sqft", col="cluster", hue='logerror_abs', col_wrap=2)

### Takeaways:
- cluster 0 shows highest log errors

### Check Significant Difference

In [ ]:
# check mean of logerror for each cluster
train.groupby(by='cluster').logerror_abs.mean()

In [ ]:
# run one sample T-Test
alpha = 0.05
cluster_logerror = train[train.cluster==2].logerror_abs
overall_logerror = train.logerror_abs.mean()

In [ ]:
t, p = stats.ttest_1samp(cluster_logerror, overall_logerror)
print(f't={t}, p={p/2}, alpha={alpha}')

# <font color = 'red'> Clustering Combo 3 (House Dollar Per Sqft, Age, Tax Rate)

In [ ]:
# Filter columsn from train scaled
cols = ['age', 'house_dollar_sqft', 'tax_rate']
train_scaled_X = train_scaled[cols]
train_scaled_X.head()

In [ ]:
sns.relplot(data=train_scaled, x='age', y='house_dollar_sqft', hue= train.logerror_abs, col='fips', kind='scatter')


In [ ]:
# Use elbow method to see if inertia values support visual exploration
# plot inertia vs k

with plt.style.context('seaborn-whitegrid'):
    plt.figure(figsize=(9, 6))
    pd.Series({k: KMeans(k, random_state=123).fit(train_scaled).inertia_ for k in range(2, 12)}).plot(marker='x')
    plt.xticks(range(2, 12))
    plt.xlabel('k')
    plt.ylabel('inertia')
    plt.title('Change in inertia as k increases')

In [ ]:
# Move forward with k=3
kmeans = KMeans(n_clusters = 4, random_state=123)
kmeans.fit(train_scaled_X)

# And assign the cluster number to a column on the dataframe
train["cluster"] = kmeans.predict(train_scaled_X)
train.head()

In [ ]:
# show house dollar per sqft for age vs log error by cluster
sns.relplot(data=train, x="age", y="house_dollar_sqft", col="cluster", hue='logerror_abs', col_wrap=2)

### Takeaways:
- cluster 3 shows highest log errors

### Check Significant Difference

In [ ]:
# check mean of logerror for each cluster
train.groupby(by='cluster').logerror_abs.mean()

In [ ]:
# run one sample T-Test
alpha = 0.05
cluster_logerror = train[train.cluster==3].logerror_abs
overall_logerror = train.logerror_abs.mean()

t, p = stats.ttest_1samp(cluster_logerror, overall_logerror)
print(f't={t}, p={p/2}, alpha={alpha}')

## Takeaway from Clustering
- Clustering 1 show largest differnce of log error

## Forward with Cluster 1

In [ ]:
# Filter columsn from train scaled
cols = ['age', 'house_dollar_sqft']
train_scaled_X = train_scaled[cols]
train_scaled_X.head()

# Move forward with k=3
kmeans = KMeans(n_clusters = 3, random_state=123)
kmeans.fit(train_scaled_X)

# And assign the cluster number to a column on the dataframe
train["cluster"] = kmeans.predict(train_scaled_X)

# <font color = 'red'> Prep for Modeling

In [ ]:
# hot one encode for cluster column using get_dummies for train, validate, test
df_dummies_train = pd.get_dummies(data=train.cluster, prefix='cluster', drop_first=True)
df_dummies_validate = pd.get_dummies(data=validate.cluster, prefix='cluster', drop_first=True)
df_dummies_test = pd.get_dummies(data=test.cluster, prefix='cluster', drop_first=True)

# concat df_dummies with train on columns
train = pd.concat([train, df_dummies_train], axis=1)
validate = pd.concat([validate, df_dummies_validate], axis=1)
test = pd.concat([test, df_dummies_test], axis=1)

train.head()

## <font color = 'red'>Select Columns

In [ ]:
# select columns to model
cols = ['land_dollar_sqft','house_dollar_sqft', 'age', 'logerror_abs']

## <font color = 'red'> Create y_train and X_train sets

In [ ]:
# establish target column
target = 'logerror_abs'

# create X & y version of train, validate, test with y the target and X are the features. 
X_train = train[cols].drop(columns=[target])
y_train = train[target]

X_validate = validate[cols].drop(columns=[target])
y_validate = validate[target]

X_test = test[cols].drop(columns=[target])
y_test = test[target]

In [ ]:
X_train.shape, X_validate.shape, X_test.shape, y_train.shape, y_validate.shape, y_test.shape

## <font color = 'red'> Scale for Modeling

In [ ]:
# Create the scale container
scaler = sklearn.preprocessing.MinMaxScaler()


# Fit the scaler to the features
scaler.fit(X_train)

# create scaled X versions 
X_train_scaled = scaler.transform(X_train)
X_validate_scaled = scaler.transform(X_validate)
X_test_scaled = scaler.transform(X_test)

# Convert numpy array to pandas dataframe for feature Engineering
X_train_scaled = pd.DataFrame(X_train_scaled, index=X_train.index, columns=X_train.columns.to_list())
X_validate_scaled = pd.DataFrame(X_validate_scaled, index=X_validate.index, columns=X_validate.columns.to_list())
X_test_scaled = pd.DataFrame(X_test_scaled, index=X_test.index, columns=X_test.columns.to_list())

In [ ]:
X_train_scaled.head()

## <font color = 'red'> Feature Engineering

### Use Select K Best to find 3 Best Features


In [ ]:
from sklearn.feature_selection import SelectKBest, f_regression

# Use f_regression stats test each column to find best 3 features
f_selector = SelectKBest(f_regression, k=3)

# find tthe best correlations with y
f_selector.fit(X_train_scaled, y_train)

# Creaet boolean mask of the selected columns. 
feature_mask = f_selector.get_support()

# get list of top K features. 
f_feature = X_train_scaled.iloc[:,feature_mask].columns.tolist()

f_feature

### Note:
- Select K Best selected area, year_built, and bathrooms.

### Use Recursive Feature Elemination to select 3 best features

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import RFE

# create the ML algorithm container
lm = LinearRegression()

# create the rfe container with the the number of features I want. 
rfe = RFE(lm, n_features_to_select=3)

# fit RFE to the data
rfe.fit(X_train_scaled,y_train)  

# get the mask of the selected columns
feature_mask = rfe.support_

# get list of the column names. 
rfe_feature = X_train_scaled.iloc[:,feature_mask].columns.tolist()

rfe_feature

### Notes:
- RFE Select 3 Best Columns: area, latitude, longitude
- RFE Selects 3 best columns:land_dollar_sqft, house_dollar_sqft, age



In [ ]:
X_train_scaled.head()

In [ ]:
# Remove Features before Modeling
# X_train_scaled = X_train_scaled.drop(columns=['pools','lot_size','bedrooms','bathrooms'])
# X_validate_scaled = X_validate_scaled.drop(columns=['pools','lot_size','bedrooms','bathrooms'])
# X_test_scaled = X_test_scaled.drop(columns=['pools','lot_size','bedrooms','bathrooms'])

# <font color = 'red'>Modeling

## Create Baseline & Evaluate with RMSE

In [ ]:
# We need y_train and y_validate to be dataframes to append the new columns with predicted values. 
y_train = pd.DataFrame(y_train)
y_validate = pd.DataFrame(y_validate)
y_test = pd.DataFrame(y_test)
y_train.head()

In [ ]:
# Add target mean column as baseline check
y_train['mean_pred'] = y_train.logerror_abs.mean()
y_validate['mean_pred'] = y_validate.logerror_abs.mean()

# add target median column as baseline check
y_train['median_pred'] = y_train.logerror_abs.median()
y_validate['median_pred'] = y_validate.logerror_abs.median()

In [ ]:
# Create Baseline RMSE of target mean
rmse_train = mean_squared_error(y_train.logerror_abs, y_train.mean_pred) ** .5
rmse_validate = mean_squared_error(y_validate.logerror_abs, y_validate.mean_pred) ** .5

In [ ]:
# Show RMSE Score for train and validate
print("RMSE using Mean\nTrain/In-Sample: ", round(rmse_train, 2), 
      "\nValidate/Out-of-Sample: ", round(rmse_validate, 2))

In [ ]:
# Create Baseline RMSE of target median
rmse_train = mean_squared_error(y_train.logerror_abs, y_train.median_pred) ** .5
rmse_validate = mean_squared_error(y_validate.logerror_abs, y_validate.median_pred) ** .5

In [ ]:
# Show RMSE Score for train and validate
print("RMSE using Median\nTrain/In-Sample: ", round(rmse_train, 2), 
      "\nValidate/Out-of-Sample: ", round(rmse_validate, 2))

### Notes:
- Mean Baseline RMSE (.03) is same as Median Baseline RMSE (.03)
- Going Forward with Mean for Baseline

### Create Metric Holder

In [ ]:
metric_df = pd.DataFrame(data=[
            {
                'model': 'mean_baseline', 
                'RMSE_train': rmse_train,
                'RMSE_validate': rmse_validate,
                'RMSE_test:': 'none',
                'RMSE_diff:': rmse_train - rmse_validate
                }
            ])

In [ ]:
metric_df

## <font color = 'red'> Develop Models 

### Create Model 1 - Linear Regression (OLS) without Clusters

In [ ]:
# create, fit, predict ols model for train and validate
ols = LinearRegression()
ols.fit(X_train_scaled, y_train.logerror_abs)
# predict for train
y_train['ols_pred_wo_cluster'] = ols.predict(X_train_scaled)
# predict validate
y_validate['ols_pred_wo_cluster'] = ols.predict(X_validate_scaled)

In [ ]:
ols.coef_

In [ ]:
y_train.head()

In [ ]:
X_train.head()

In [ ]:
# evaluate rmse of train
rmse_train = mean_squared_error(y_train.logerror_abs, y_train.ols_pred_wo_cluster) ** .5

# evaluate rmse of validate
rmse_validate = mean_squared_error(y_validate.logerror_abs, y_validate.ols_pred_wo_cluster) ** .5

In [ ]:
print("RMSE for OLS using LinearRegression\nTraining/In-Sample: ", rmse_train, 
      "\nValidation/Out-of-Sample: ", rmse_validate)

In [ ]:
# add to eval to metric holder
metric_df = metric_df.append({
    'model': 'ols_egressor_wo_cluster', 
    'RMSE_train': rmse_train,
    'RMSE_validate': rmse_validate,
    'RMSE_test:': 'none',
    'RMSE_diff:': rmse_train - rmse_validate
    }, ignore_index=True)

In [ ]:
metric_df

In [ ]:
y_train.head()

### Create Model 2: Linear Regression (OLS) with Cluster Columns

In [ ]:
# select columns to model including cluster columns
cols = ['land_dollar_sqft','house_dollar_sqft', 'age', 'logerror_abs', 'cluster_1', 'cluster_2']

In [ ]:
# establish target column
target = 'logerror_abs'

# create X & y version of train, validate, test with y the target and X are the features. 
X_train = train[cols].drop(columns=[target])

X_validate = validate[cols].drop(columns=[target])

X_test = test[cols].drop(columns=[target])

In [ ]:
# Create the scale container
scaler = sklearn.preprocessing.MinMaxScaler()


# Fit the scaler to the features
scaler.fit(X_train)

# create scaled X versions 
X_train_scaled = scaler.transform(X_train)
X_validate_scaled = scaler.transform(X_validate)
X_test_scaled = scaler.transform(X_test)

# Convert numpy array to pandas dataframe for feature Engineering
X_train_scaled = pd.DataFrame(X_train_scaled, index=X_train.index, columns=X_train.columns.to_list())
X_validate_scaled = pd.DataFrame(X_validate_scaled, index=X_validate.index, columns=X_validate.columns.to_list())
X_test_scaled = pd.DataFrame(X_test_scaled, index=X_test.index, columns=X_test.columns.to_list())

In [ ]:
# create, fit, predict ols model for train and validate
ols = LinearRegression()
ols.fit(X_train_scaled, y_train.logerror_abs)

# predict on train
y_train['ols_pred_w_cluster'] = ols.predict(X_train_scaled)

# predict validate
y_validate['ols_pred_w_cluster'] = ols.predict(X_validate_scaled)

In [ ]:
# evaluate rmse for train
rmse_train = mean_squared_error(y_train.logerror_abs, y_train.ols_pred_w_cluster) ** .5

# evaluate rmse for validate
rmse_validate = mean_squared_error(y_validate.logerror_abs, y_validate.ols_pred_w_cluster) ** .5

In [ ]:
print("RMSE for OLS using LinearRegression\nTraining/In-Sample: ", rmse_train, 
      "\nValidation/Out-of-Sample: ", rmse_validate)

In [ ]:
# add to eval to metric holder
metric_df = metric_df.append({
    'model': 'ols_egressor_w_cluster', 
    'RMSE_train': rmse_train,
    'RMSE_validate': rmse_validate,
    'RMSE_test:': 'none',
    'RMSE_diff:': rmse_train - rmse_validate
    }, ignore_index=True)

In [ ]:
metric_df

### Takeaways:
- With clusters did worse

###  Create Model 3: Linear Regression (OLS) with Clusters and More Features

In [ ]:
# select columns to model including cluster columns
cols = ['land_dollar_sqft','house_dollar_sqft', 'age', 'logerror_abs', 'cluster_1', 'cluster_2', 'longitude',
       'latitude','bed_bath_ration','tax_rate']

In [ ]:
# establish target column
target = 'logerror_abs'

# create X & y version of train, validate, test with y the target and X are the features. 
X_train = train[cols].drop(columns=[target])

X_validate = validate[cols].drop(columns=[target])

X_test = test[cols].drop(columns=[target])

In [ ]:
# Create the scale container
scaler = sklearn.preprocessing.MinMaxScaler()


# Fit the scaler to the features
scaler.fit(X_train)

# create scaled X versions 
X_train_scaled = scaler.transform(X_train)
X_validate_scaled = scaler.transform(X_validate)
X_test_scaled = scaler.transform(X_test)

# Convert numpy array to pandas dataframe for feature Engineering
X_train_scaled = pd.DataFrame(X_train_scaled, index=X_train.index, columns=X_train.columns.to_list())
X_validate_scaled = pd.DataFrame(X_validate_scaled, index=X_validate.index, columns=X_validate.columns.to_list())
X_test_scaled = pd.DataFrame(X_test_scaled, index=X_test.index, columns=X_test.columns.to_list())

In [ ]:
# create, fit, predict ols model for train and validate
ols = LinearRegression()
ols.fit(X_train_scaled, y_train.logerror_abs)

# predict on train
y_train['ols_pred_w_cluster_and_features'] = ols.predict(X_train_scaled)

# predict validate
y_validate['ols_pred_w_cluster_and_features'] = ols.predict(X_validate_scaled)

In [ ]:
# evaluate rmse for train
rmse_train = mean_squared_error(y_train.logerror_abs, y_train.ols_pred_w_cluster_and_features) ** .5

# evaluate rmse for validate
rmse_validate = mean_squared_error(y_validate.logerror_abs, y_validate.ols_pred_w_cluster_and_features) ** .5

In [ ]:
print("RMSE for OLS using LinearRegression\nTraining/In-Sample: ", rmse_train, 
      "\nValidation/Out-of-Sample: ", rmse_validate)

In [ ]:
# add to eval to metric holder
metric_df = metric_df.append({
    'model': 'ols_egressor_w_cluster_and_features', 
    'RMSE_train': rmse_train,
    'RMSE_validate': rmse_validate,
    'RMSE_test:': 'none',
    'RMSE_diff:': rmse_train - rmse_validate
    }, ignore_index=True)

In [ ]:
metric_df

### Takeaways:
- ???

### Create Model 4 - Polynomial Regression with Clusters and More Features

In [ ]:
# create polynomial features
pf = PolynomialFeatures(degree=2)

# fit and transform X_train_scaled
X_train_degree2 = pf.fit_transform(X_train_scaled)

# transform X_validate_scaled & X_test_scaled to new sets
X_validate_degree2 = pf.transform(X_validate_scaled)
X_test_degree2 =  pf.transform(X_test_scaled)

In [ ]:
# create the model object
osl2 = LinearRegression()

# fit the model train data. Specify y_train columns since it was converted to dataframe  
osl2.fit(X_train_degree2, y_train.logerror_abs)

# predict train
y_train['pr_pred'] = osl2.predict(X_train_degree2)

# create rmse
rmse_train = mean_squared_error(y_train.logerror_abs, y_train.pr_pred) ** .5

# predict validate
y_validate['pr_pred'] = osl2.predict(X_validate_degree2)

# evaluate rmse
rmse_validate = mean_squared_error(y_validate.logerror_abs, y_validate.pr_pred) ** .5

In [ ]:
print("RMSE for Polynomial Model, degrees=2\nTraining/In-Sample: ", rmse_train, 
      "\nValidation/Out-of-Sample: ", rmse_validate)

In [ ]:
# add to metric holder
metric_df = metric_df.append({
    'model': 'PolynomialRegressor', 
    'RMSE_train': rmse_train,
    'RMSE_validate': rmse_validate,
    'RMSE_test:': 'none',
    'RMSE_diff:': rmse_train - rmse_validate
    }, ignore_index=True)

In [ ]:
metric_df

## <font color = 'red' > Evaluate
    
### Plotting Actual vs Predicted Values

In [ ]:
plt.figure(figsize=(16,8))

#Basline
plt.plot(y_validate.logerror_abs, y_validate.mean_pred, alpha=.5, color="gray", label='_nolegend_')
plt.annotate("Baseline: Predict Using Mean", (0,.034))

# Ideal Line
plt.plot(y_validate.logerror_abs, y_validate.logerror_abs, alpha=.5, color="blue", label='_nolegend_')
plt.annotate("The Ideal Line: Predicted = Actual", (0.005, 0), rotation=27)

# Model 1: OLS without Clusters
plt.scatter(y_validate.logerror_abs, y_validate.ols_pred_wo_cluster, 
            alpha=.5, color="red", s=10, label="Model 1: OLS wo Clusters")

# Model 2: OlS with Clusters
plt.scatter(y_validate.logerror_abs, y_validate.ols_pred_w_cluster, 
            alpha=.5, color="yellow", s=10, label="Model 2: OLS w Clusters")

# Model 3: OLS with Clusters and More Features
plt.scatter(y_validate.logerror_abs, y_validate.ols_pred_w_cluster_and_features, 
            alpha=.5, color="green", s=10, label="Model 3: OLS w Clusters and More Features")

# Model 4: Polynomial Regresion with Clusters and More Features
plt.scatter(y_validate.logerror_abs, y_validate.pr_pred, 
            alpha=.5, color="brown", s=10, label="Model 4: Poly Regression with Clusters and More Features")



plt.legend()
plt.xlabel("Actual Tax Value")
plt.ylabel("Predicted Tax Value")
# plt.title("Where are predictions more extreme? More modest?")
# plt.annotate("The polynomial model appears to overreact to noise", (2.0, -10))
# plt.annotate("The OLS model (LinearRegression)\n appears to be most consistent", (15.5, 3))
plt.show()

### Notes:
- ????

### Residual Plots: Plotting the Errors in Predictions

In [ ]:
# y_validate.head()
plt.figure(figsize=(16,8))
plt.axhline(label="No Error")
plt.annotate("Line of No Error", (0, -.01))
plt.scatter(y_validate.logerror_abs, y_validate.ols_pred_wo_cluster - y_validate.logerror_abs, 
            alpha=.5, color="red", s=10, label="Model: OLS without Cluster")
plt.scatter(y_validate.logerror_abs, y_validate.ols_pred_w_cluster - y_validate.logerror_abs, 
            alpha=.5, color="yellow", s=10, label="Mode2: OLS with Cluster")
plt.scatter(y_validate.logerror_abs, y_validate.ols_pred_w_cluster_and_features - y_validate.logerror_abs, 
            alpha=.5, color="green", s=10, label="Model 3: OLS with Cluster and More Features")
plt.scatter(y_validate.logerror_abs, y_validate.pr_pred - y_validate.logerror_abs, 
            alpha=.5, color="green", s=10, label="Model 4: Poly Regressor with Cluster and More Features")
plt.legend()
plt.xlabel("Actual Tax Value")
plt.ylabel("Residual/Error: Predicted Tax Value - Actual Tax Value")
# plt.title("Do the size of errors change as the actual value changes?")
# plt.annotate("The polynomial model appears to overreact to noise", (2.0, -10))
# plt.annotate("The OLS model (LinearRegression)\n appears to be most consistent", (15.5, 3))
plt.show()

### Notes:
- ???

### Histograms of Actuals vs Predicted Values

In [ ]:
# plot to visualize actual vs predicted. 
plt.figure(figsize=(16,8))
plt.hist(y_validate.logerror_abs, color='blue', alpha=.5, label="Absolute Value of Log Errors")
plt.hist(y_validate.ols_pred_wo_cluster, color='red', alpha=.5, label="Model 1: OLS without Clusters")
plt.hist(y_validate.ols_pred_w_cluster, color='yellow', alpha=.5, label="Model2: OLS with Clusters")
plt.hist(y_validate.ols_pred_w_cluster_and_features, color='green', alpha=.5, label="Model 3: OLS with Cluster and More Features")
plt.hist(y_validate.pr_pred, color='green', alpha=.5, label="Model 4: Poly Regressor with Clusters and More Features")
plt.xlabel("ABS LogError Values")
plt.ylabel("Number of Homes ")
plt.title("Comparing the Distribution of Actual ABS Log Error to Distributions of Predicted ABS Log Error for Models")
plt.legend()
plt.show()

### Notes:
- Models capture most of actual values distribution but miss the lower valued homes

## Model Selection and Out-of-Sample Evaluatoin

In [ ]:
# predict train
y_test['pr_pred'] = osl2.predict(X_test_degree2)

# create rmse
rmse_test = mean_squared_error(y_test.logerror_abs, y_test.pr_pred) ** .5

print("RMSE for Polynomial Model, degrees=2\nTest/Out-of-Sample: ", rmse_test)# add to metric holder

In [ ]:
# add to metric holder
metric_df = metric_df.append({
    'model': 'PolynomialRegressor', 
    'RMSE_train': 'none',
    'RMSE_validate': 'none',
    'RMSE_test:': rmse_test,
    'RMSE_diff:': rmse_train - rmse_test
    }, ignore_index=True)

In [ ]:
metric_df

### Notes:
- Polynomial Model RMSE is 18.6M indicating wrong model to use and/or overfit

# <font color = 'red'> Conclusion

## Summary
- 

## Recommendations
- ???

## Conclusion
- ???

### Project Description

- A model that predicts property tax assessed values of single family properties based on 2017 transactions.
- Identify ways to make a better model for Zillow.
- Create new features out of existing features.
- Try non-linear regression model or try a model for each count

Deliver:
1. Github Repo
    - README.md
    - Final Report
    - Working Notebooks
    - Acquire and Prepare Modules
    - Misc Artifacts
1. Regression Model (Linear or Non-Linear)
2. Final Notebook with Recommendation
    - 4 Visuals, 2 with stats tests
        - Data Context of Target Variable (Distribution of values)
    - 3 Best Models (show steps/code to fit/eval/select)
    - Visual of how best Model performed
3. Working Notebook:
    - Reproducible comments
    - Exploration with at least two stats test during exploration
5. Presentation
    - Findings
    - Methodologies
    - Conclusions

### Initial Questions

- What are the transactions are in 2017?
- What states and counties are the properties located in?
- What is the tax rate per county or fips? (might have to combine columns to calcualate)
- What is the distribution of tax rates for each county?
- What is the distribution of taxes across fips?
- What are the drivers of single family property values?
- Why do some properties have a much higher value than others when they are located so close to each other?
- Why are some properties valued so differently from others when they have nearly the same physical attributes but only differ in location? 
- Is having 1 bathroom worse than having 2 bedrooms?